In [1]:
#pragma cling add_library_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/")
#pragma cling add_include_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/include")
#pragma cling add_include_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/include/torch/csrc/api/include/")

In [2]:
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libc10.so")
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libgomp-75eea7e8.so.1")
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libtorch.so")

In [3]:
#include <torch/torch.h>

In [4]:
torch::Tensor Flatten(torch::Tensor x){
    return x.view({x.sizes()[0], -1});
}

In [16]:
struct AlexNet: torch::nn::Module {
    AlexNet()
        : conv1(torch::nn::Conv2dOptions(1, 6, 3)), 
    conv2(torch::nn::Conv2dOptions(6, 16, 3)), 
    fc1(576, 120), 
    fc2(120, 84), fc3(84, 10){
        register_module("conv1", conv1);
        register_module("conv2", conv2);
        register_module("fc1", fc1);
        register_module("fc2", fc2);
        register_module("fc3", fc3);
    }
    
    torch::Tensor forward(torch::Tensor x){
        x = torch::max_pool2d(torch::relu(conv1->forward(x)), 2);
        x = torch::max_pool2d(torch::relu(conv2->forward(x)), 2);
        x = Flatten(x);
        x = torch::relu(fc1->forward(x));
        x = torch::relu(fc2->forward(x));
        x = fc3->forward(x);
        return x;
    }
    
    torch::nn::Conv2d conv1;
    torch::nn::Conv2d conv2;
    torch::nn::Linear fc1;
    torch::nn::Linear fc2;
    torch::nn::Linear fc3;
}

In [17]:
AlexNet model;

In [18]:
torch::Tensor ip = torch::randn({1, 1, 32, 32});
auto output = model.forward(ip);

In [19]:
std::cout << output;

 0.0331  0.0509 -0.0283 -0.1149  0.0833  0.1142  0.0779  0.0514 -0.0137 -0.0771
[ CPUFloatType{1,10} ]

In [20]:
void train(){
    
}

In [ ]:
void test(){
    
}